#  Evolutionary Multiple-objective Optimization (part for 4.0)

- This script is for those who want to improve their final grade from 3.0 to 4.0. 
- Your task is to implement any one evolutionary algorithm for multiple-objective optimization introduced during the lecture (NSGA/NSGA-II/NSGA-II/MOEA/D/IBEA/HypE/SPEA2). It is recommended to implement NSGA-II or MOEA/D. 
- Note that it has to be your implementation (using external libraries is forbidden).
- The problem to be solved is the portfolio optimization tackled during lab 1.
- You can use the same data and price predictions as you made for lab 1 (Bundle1.zip) or update them accordingly to the next stage if you participate in the portfolio game (it is up to you).
- Apart from the two-objective scenario, consider also the three-objective one. As for the third objective, think about some reasonable risk-measure. You can maximize the number of non-zero weights, which should refer to minimizing risk by diversifying investments.
- Perform experimental evaluation of your implementation. You can use, e.g., the IDG or the HV metric to quantify the quality of populations constructed by the method.
- The experimental evaluation should be "reasonably extensive." E.g., run your method multiple times and average the results, show average convergence plots, do the sensitivity analysis (just four combinations of population size/generations will be enough), and depict some final populations. Also, compare the populations (only for 2D scenarios) with those generated by the ECM or WSM algorithm. Note that ECM and WSM already generate Pareto optimal solutions, so these can be considered good benchmarks for comparison.
- You can report your results here, i.e., in the jupyter notebook. You do not need to prepare any pdf report, etc. 

In [31]:
import numpy as np

from helpers.helper import generate_risk_and_return_weights

class Instance:
    def __init__(self, cov_matrix, predictions):
        self.cov_matrix = cov_matrix
        self.predictions = predictions
        self.assets_number = len(predictions)

class ScalarizingFunction:
    def __init__(self, risk_factor, profit_factor):
        self.risk_factor = risk_factor
        self.profit_factor = profit_factor

    def __call__(self, individual, instance):
        return self.profit_factor * instance.predictions.dot(individual) - self.risk_factor * individual.dot(instance.cov_matrix).dot(individual)

    def __eq__(self, other):
        if isinstance(other, ScalarizingFunction):
            return (self.risk_factor, self.profit_factor) == (other.risk_factor, other.profit_factor)
        return False

    def __hash__(self):
        return hash((self.risk_factor, self.profit_factor))

# class Individual:
#     def __init__(self, weights):
#         self.weights = weights


class MOEADAlgorithm:
    def __init__(self, instance, **kwargs):
        self.instance = instance

        self.population_size = kwargs.get('population_size', 10)
        self.generations = kwargs.get('generations', 100)
        self.mating_pool_size = kwargs.get('mating_pool_size', 10)
        self.neighborhood_size = kwargs.get('neighborhood_size', 5)

        self.population = []
        self.scalarizing_functions = []
        self.function_to_incumbents = {}

    def run(self):
        self.__generate_scalarizing_functions()
        self.population = self.__initialize_population(len(self.scalarizing_functions))
        self.__random_assignment()
        for i in range(self.generations):
            self.population = self.__evolve_population()

        return set(self.function_to_incumbents.values())

    def __generate_scalarizing_functions(self, n = 100):
        profit_risk_factors = generate_risk_and_return_weights(n)
        for profit_factor, risk_factor in profit_risk_factors:
            self.scalarizing_functions.append(ScalarizingFunction(risk_factor, profit_factor))

    def __initialize_population(self, n):
        return np.array([self._generate_random_individual() for _ in range(n)])

    def _generate_random_individual(self):
        return np.random.rand(self.instance.assets_number)
        # return Individual(np.random.rand(self.instance.assets_number))

    def __random_assignment(self):
        for scalarizing_function in self.scalarizing_functions:
            self.function_to_incumbents[scalarizing_function] = np.random.choice(self.population, replace=False)

    def __evolve_population(self):
        scalarizing_function = list(self.function_to_incumbents.keys())
        np.random.shuffle(scalarizing_function)

        for scalarizing_function in self.scalarizing_functions:
            mating_pool = self.__create_mating_pool(scalarizing_function)
            parents = self.__select_parents(mating_pool)
            child = self.__crossover(parents)
            self.__update_incumbents(child, scalarizing_function, mating_pool)

    def __create_mating_pool(self, scalarizing_function):
        profit_factor, risk_factor = scalarizing_function.profit_factor, scalarizing_function.risk_factor
        most_similar_functions = sorted(self.scalarizing_functions, key=lambda f: abs(f.profit_factor - profit_factor) + abs(f.risk_factor - risk_factor))[:self.neighborhood_size]
        return most_similar_functions

    def __select_parents(self, mating_pool):
        mating_pool = list(mating_pool.items())
        np.random.shuffle(mating_pool)
        return mating_pool[:2]

    def __crossover(self, parents):
        parent1, parent2 = parents
        crossover_point = np.random.randint(0, self.instance.assets_number)
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        return child

    def __update_incumbents(self, child, child_function, mating_pool):
        for function, incumbent in mating_pool:
            if child_function(child, self.instance) < function(incumbent, self.instance):
                self.function_to_incumbents[function] = child